# Model Summary
- 

## Experiment Methodology and Findings

## Testing TODO
1. SST Basic model
2. Try BERT, RoBERTa, and XNet

In [15]:
# Drive Set Up

from google.colab import drive
drive.mount("/content/drive", force_remount=True)
HW2_PATH = "stanford/xcs224u"

# Add HW2_PATH to the Colab VM'a Python interpreter.
import sys
sys.path.append("/content/drive/MyDrive/{0}".format(HW2_PATH))
#sys.path.append("/content/drive/My Drive/{0}".format(HW2_PATH))

Mounted at /content/drive


## Install and Import Dependencies
 

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 5.3 MB/s 
     |████████████████████████████████| 636 kB 47.6 MB/s 
     |████████████████████████████████| 3.3 MB 46.9 MB/s 
     |████████████████████████████████| 895 kB 46.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [16]:
import os
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
from transformers import BertConfig, BertModel, BertTokenizer
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer
from transformers import XLNetConfig, XLNetModel, XLNetTokenizer

from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from torch_rnn_classifier import TorchRNNModel
from torch_rnn_classifier import TorchRNNClassifier
from torch_rnn_classifier import TorchRNNClassifierModel
from torch_rnn_classifier import TorchRNNClassifier

import sst
import utils

import pandas as pd
import json

## Training Data

In [17]:
SST_PATH = os.path.join(sys.path[-1], 'data/sentiment')
DYNASENT_PATH = os.path.join(sys.path[-1], 'data/dynasent/dynasent-v1.1')
TWITTER_PATH = os.path.join(sys.path[-1], 'data/twitter')

In [18]:
def load_dynasent_dataset(*src_filenames, labels=None):
    data = []
    for filename in src_filenames:
        with open(filename) as f:
            for line in f:
                d = json.loads(line)
                if labels is None or d['gold_label'] in labels:
                    data.append(d)
    return data

In [19]:
utils.fix_random_seeds()

bakeoff_dev = sst.bakeoff_dev_reader(SST_PATH)
sst_train = sst.train_reader(SST_PATH, include_subtrees=True, dedup=True)
sst_dev = sst.dev_reader(SST_PATH)

dynasent_train = load_dynasent_dataset(os.path.join(DYNASENT_PATH, 'dynasent-v1.1-round01-yelp-train.jsonl'))
dynasent_dev = load_dynasent_dataset(os.path.join(DYNASENT_PATH, 'dynasent-v1.1-round01-yelp-dev.jsonl'))
# TODO: Read round02 too

In [20]:
# Process datasets
# TODO: Add cell above for processing examples from the dynasent dataset. 
train_df = sst_train

## Models

# TODO
- BERT
  - Try bert-based-uncased, bert-large-uncased, bert-large-cased
- RoBERTa
  - Try roberta-large, roberta-large-mnli
  - Different classivier?
  - Different TorchShallowNN?
- XLNet

# DEL: FOR TESTING PURPOSES ONLY

## DEL: Test RoBERTa baseline, sst_train only

In [33]:
class MyRobertaClassifierModel(nn.Module):
    def __init__(self, n_classes, weights_name='roberta-base'):
        super().__init__()
        self.n_classes = n_classes
        self.weights_name = weights_name
        self.model = RobertaModel.from_pretrained(self.weights_name)
        self.model.train()
        self.hidden_dim = self.model.embeddings.word_embeddings.embedding_dim
        # The only new parameters -- the classifier:
        self.classifier_layer = nn.Linear(
            self.hidden_dim, self.n_classes)

    def forward(self, indices, mask):
        reps = self.model(
            indices, attention_mask=mask)
        return self.classifier_layer(reps.pooler_output)

class MyRobertaClassifier(TorchShallowNeuralClassifier):
    def __init__(self, weights_name, *args, **kwargs):
        self.weights_name = weights_name
        self.tokenizer = RobertaTokenizer.from_pretrained(self.weights_name)
        super().__init__(*args, **kwargs)
        self.params += ['weights_name']

    def build_graph(self):
        return MyRobertaClassifierModel(self.n_classes_, self.weights_name)

    def build_dataset(self, X, y=None):
        data = self.tokenizer.batch_encode_plus(
            X,
            max_length=None,
            add_special_tokens=True,
            padding='longest',
            return_attention_mask=True)
        indices = torch.tensor(data['input_ids'])
        mask = torch.tensor(data['attention_mask'])
        if y is None:
            dataset = torch.utils.data.TensorDataset(indices, mask)
        else:
            self.classes_ = sorted(set(y))
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            y = [class2index[label] for label in y]
            y = torch.tensor(y)
            dataset = torch.utils.data.TensorDataset(indices, mask, y)
        return dataset

def roberta_fine_tune_phi(text):
    return text

# This was first written as a version containing hyperparameter search
# and modified to use the final best version from the search
def fit_my_roberta_classifier(X, y):
    mod = MyRobertaClassifier(
        gradient_accumulation_steps=8,
        eta=0.005,
        hidden_dim=300,
        weights_name='roberta-base', 
        batch_size=16,  # Small batches to avoid memory overload.
        max_iter=10,  # We'll search based on 1 iteration for efficiency.
        n_iter_no_change=5,   # Early-stopping params are for the
        early_stopping=True)  # final evaluation.

    mod.fit(X, y)

    return mod

In [ ]:
# took around an hour maybe to run
# without gpu: still has not completed after 5h
roberta_classifier_xval = sst.experiment(
    train_df,
    roberta_fine_tune_phi,
    fit_my_roberta_classifier,
    assess_dataframes=[sst_dev, bakeoff_dev],
    vectorize=False)  # Pass in the hidden state directly

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Finished epoch 4 of 10; error is 1223.9074290394783

First run, with batch size 8, max_iter 1, eta 0.0001

```
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Finished epoch 1 of 1; error is 1466.5560857672244Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.739     0.914     0.817       428
     neutral      0.535     0.236     0.327       229
    positive      0.790     0.838     0.813       444

    accuracy                          0.742      1101
   macro avg      0.688     0.662     0.653      1101
weighted avg      0.717     0.742     0.714      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.394     0.887     0.545       565
     neutral      0.709     0.403     0.514      1019
    positive      0.809     0.530     0.641       777

    accuracy                          0.561      2361
   macro avg      0.637     0.607     0.567      2361
weighted avg      0.666     0.561     0.563      2361

```

In [ ]:
optimized_roberta_classifier = roberta_classifier_xval['model']

# Remove the rest of the experiment results to clear out some memory
#del roberta_classifier_xval

In [ ]:
def fit_optimized_my_roberta_classifier(X, y):
    optimized_roberta_classifier.max_iter = 1000
    optimized_roberta_classifier.fit(X, y)
    return optimized_roberta_classifier

In [ ]:
# Takes as long time to run
roberta_experiment = sst.experiment(
    train_df, 
    roberta_fine_tune_phi,
    fit_optimized_my_roberta_classifier,
    assess_dataframes=[sst_dev, bakeoff_dev],
    vectorize=False)  # Pass in the hidden state directly

## DEL: XLNet Test

In [30]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [28]:
class MyXLNetClassifierModel(nn.Module):
    def __init__(self, n_classes, weights_name='xlnet-base'):
        super().__init__()
        self.n_classes = n_classes
        self.weights_name = weights_name
        self.model = XLNetModel.from_pretrained(self.weights_name)
        self.model.train()
        self.hidden_dim = self.model.embeddings.word_embeddings.embedding_dim
        # The only new parameters -- the classifier:
        self.classifier_layer = nn.Linear(
            self.hidden_dim, self.n_classes)

    def forward(self, indices, mask):
        reps = self.model(
            indices, attention_mask=mask)
        return self.classifier_layer(reps.pooler_output)

class MyXLNetClassifier(TorchShallowNeuralClassifier):
    def __init__(self, weights_name, *args, **kwargs):
        self.weights_name = weights_name
        self.tokenizer = XLNetTokenizer.from_pretrained(self.weights_name)
        super().__init__(*args, **kwargs)
        self.params += ['weights_name']

    def build_graph(self):
        return MyXLNetClassifierModel(self.n_classes_, self.weights_name)

    def build_dataset(self, X, y=None):
        data = self.tokenizer.batch_encode_plus(
            X,
            max_length=None,
            add_special_tokens=True,
            padding='longest',
            return_attention_mask=True)
        indices = torch.tensor(data['input_ids'])
        mask = torch.tensor(data['attention_mask'])
        if y is None:
            dataset = torch.utils.data.TensorDataset(indices, mask)
        else:
            self.classes_ = sorted(set(y))
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            y = [class2index[label] for label in y]
            y = torch.tensor(y)
            dataset = torch.utils.data.TensorDataset(indices, mask, y)
        return dataset

def xlnet_fine_tune_phi(text):
    return text

# This was first written as a version containing hyperparameter search
# and modified to use the final best version from the search
def fit_my_xlnet_classifier(X, y):
    mod = MyXLNetClassifier(
        gradient_accumulation_steps=8,
        eta=0.0001,
        hidden_dim=300,
        weights_name='xlnet-base-cased', 
        batch_size=8,  # Small batches to avoid memory overload.
        max_iter=1,  # We'll search based on 1 iteration for efficiency.
        n_iter_no_change=5,   # Early-stopping params are for the
        early_stopping=True)  # final evaluation.

    mod.fit(X, y)

    return mod

In [32]:
# took around an hour maybe to run
# without gpu: still has not completed after 5h
xlnet_classifier_xval = sst.experiment(
    train_df,
    xlnet_fine_tune_phi,
    fit_my_xlnet_classifier,
    assess_dataframes=[sst_dev, bakeoff_dev],
    vectorize=False)  # Pass in the hidden state directl

AttributeError: ignored

## DEL: Bert baseline test, sst_train only

In [ ]:
class HfBertClassifierModel(nn.Module):
    def __init__(self, n_classes, weights_name='bert-base-cased'):
        super().__init__()
        self.n_classes = n_classes
        self.weights_name = weights_name
        self.bert = BertModel.from_pretrained(self.weights_name)
        self.bert.train()
        self.hidden_dim = self.bert.embeddings.word_embeddings.embedding_dim
        # The only new parameters -- the classifier:
        self.classifier_layer = nn.Linear(
            self.hidden_dim, self.n_classes)

    def forward(self, indices, mask):
        reps = self.bert(
            indices, attention_mask=mask)
        return self.classifier_layer(reps.pooler_output)

class HfBertClassifier(TorchShallowNeuralClassifier):
    def __init__(self, weights_name, *args, **kwargs):
        self.weights_name = weights_name
        self.tokenizer = BertTokenizer.from_pretrained(self.weights_name)
        super().__init__(*args, **kwargs)
        self.params += ['weights_name']

    def build_graph(self):
        return HfBertClassifierModel(self.n_classes_, self.weights_name)

    def build_dataset(self, X, y=None):
        data = self.tokenizer.batch_encode_plus(
            X,
            max_length=None,
            add_special_tokens=True,
            padding='longest',
            return_attention_mask=True)
        indices = torch.tensor(data['input_ids'])
        mask = torch.tensor(data['attention_mask'])
        if y is None:
            dataset = torch.utils.data.TensorDataset(indices, mask)
        else:
            self.classes_ = sorted(set(y))
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            y = [class2index[label] for label in y]
            y = torch.tensor(y)
            dataset = torch.utils.data.TensorDataset(indices, mask, y)
        return dataset

def bert_fine_tune_phi(text):
    return text

# This was first written as a version containing hyperparameter search
# and modified to use the final best version from the search
def fit_hf_bert_classifier(X, y):
    mod = HfBertClassifier(
        gradient_accumulation_steps=8,
        eta=0.0001,
        hidden_dim=300,
        weights_name='bert-base-cased', # also try bert-based-uncased
        batch_size=8,  # Small batches to avoid memory overload.
        max_iter=1,  # We'll search based on 1 iteration for efficiency.
        n_iter_no_change=5,   # Early-stopping params are for the
        early_stopping=True)  # final evaluation.

    mod.fit(X, y)

    return mod

In [ ]:
# took around an hour maybe to run
# without gpu: still has not completed after 5h
bert_classifier_xval = sst.experiment(
    train_df,
    bert_fine_tune_phi,
    fit_hf_bert_classifier,
    assess_dataframes=[sst_dev, bakeoff_dev],
    vectorize=False)  # Pass in the BERT hidden state directly!

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Finished epoch 1 of 1; error is 1385.013799475506

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.764     0.853     0.806       428
     neutral      0.444     0.275     0.340       229
    positive      0.786     0.851     0.817       444

    accuracy                          0.732      1101
   macro avg      0.664     0.660     0.654      1101
weighted avg      0.706     0.732     0.713      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.512     0.727     0.601       565
     neutral      0.664     0.674     0.669      1019
    positive      0.786     0.529     0.632       777

    accuracy                          0.639      2361
   macro avg      0.654     0.644     0.634      2361
weighted avg      0.668     0.639     0.641      2361

Mean of macro-F1 scores: 0.644


### Cell output
```
Finished epoch 1 of 1; error is 1385.013799475506Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.764     0.853     0.806       428
     neutral      0.444     0.275     0.340       229
    positive      0.786     0.851     0.817       444

    accuracy                          0.732      1101
   macro avg      0.664     0.660     0.654      1101
weighted avg      0.706     0.732     0.713      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.512     0.727     0.601       565
     neutral      0.664     0.674     0.669      1019
    positive      0.786     0.529     0.632       777

    accuracy                          0.639      2361
   macro avg      0.654     0.644     0.634      2361
weighted avg      0.668     0.639     0.641      2361

Mean of macro-F1 scores: 0.644
```

In [ ]:
optimized_bert_classifier = bert_classifier_xval['model']

# Remove the rest of the experiment results to clear out some memory
#del bert_classifier_xval

In [ ]:
def fit_optimized_hf_bert_classifier(X, y):
    optimized_bert_classifier.max_iter = 1000
    optimized_bert_classifier.fit(X, y)
    return optimized_bert_classifier

In [ ]:
# took 3.5h to run....
hfbert_experiment = sst.experiment(
    train_df, 
    bert_fine_tune_phi,
    fit_optimized_hf_bert_classifier,
    assess_dataframes=[sst_dev, bakeoff_dev],
    vectorize=False)  # Pass in the BERT hidden state directly!

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Finished epoch 7 of 1000; error is 755.2791289871093

In [ ]:
def predict_one_rnn_bert(text):
    # List of tokenized examples:
    X = [hfbert_experiment['phi'](text)]
    # Standard `predict` step on a list of lists of str:
    preds = hfbert_experiment['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

def create_bakeoff_submission_bert(
        predict_one_func,
        output_filename='cs224u-sentiment-bakeoff-entry-v1bert.csv'):

    bakeoff_test = sst.bakeoff_test_reader(SST_PATH)
    sst_test = sst.test_reader(SST_PATH)
    bakeoff_test['dataset'] = 'bakeoff'
    sst_test['dataset'] = 'sst3'
    df = pd.concat((bakeoff_test, sst_test))

    df['prediction'] = df['sentence'].apply(predict_one_func)

    df.to_csv(output_filename, index=None)

In [ ]:
create_bakeoff_submission_bert(predict_one_rnn_bert)